# Access AWS CESM2 using the AWS open data origin data 

In [1]:
# Display output of plots directly in Notebook
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")

import intake
import numpy as np
import pandas as pd
import xarray as xr
import re
# import nest_asyncio
# nest_asyncio.apply()
import aiohttp

In [2]:
import fsspec.implementations.http as fshttp
from pelicanfs.core import PelicanFileSystem, PelicanMap, OSDFFileSystem 

In [5]:
import dask 
from dask_jobqueue import SLURMCluster
from dask.distributed import Client
from dask.distributed import performance_report

In [8]:
#
rda_url    =  'https://data.rda.ucar.edu/'
intake_url = rda_url + 'harshah/intake_catalogs/cesm2-lens-osdf/aws-cesm2-le.json'

In [7]:
# # Spin up SLURM cluster
# job_extra = ['--qos=cf_lowprio','--account=ac_cumulus'] 
# cluster =  SLURMCluster(queue="cf1", cores=5, walltime='6:00:00', 
#                 local_directory='/global/scratch/users/harsha/dask_space/', 
#                 log_directory='/global/scratch/users/harsha/dask_space/', 
#                 job_extra_directives=job_extra, interface='eth0', memory="192GB") 
# client  = Client(cluster) 
# cluster.scale_up(4)

### Access the data from the AWS bucket using intake

In [9]:
osdf_catalog = intake.open_esm_datastore(new_intake_url)
osdf_catalog

,unique
Unnamed: 0,322
variable,53
long_name,51
component,4
experiment,2
forcing_variant,2
frequency,3
vertical_levels,3
spatial_domain,3
units,20
